In [132]:
# Import dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn as skl
import tensorflow as tf

In [133]:
# Import the input dataset
charity_df = pd.read_csv("Resource/charity_data.csv")
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [134]:
# Clean up dataset
charity_df = charity_df.drop(columns=['NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis = 1)
charity_df

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,996009318,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,996010315,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,996012607,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,996015768,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [135]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
charity_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [136]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
USE_CASE             5
ORGANIZATION         4
INCOME_AMT           9
dtype: int64

In [137]:
# Bin classification
classfication_counts = charity_df.CLASSIFICATION.value_counts()

# Determine which values to replace
replace_classfication = list(classfication_counts[classfication_counts < 700].index)

# Replace in DataFrame
for classification in replace_classfication:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification,"Other-classification")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000                   17326
C2000                    6074
C1200                    4837
C3000                    1918
C2100                    1883
Other-classification     1484
C7000                     777
Name: CLASSIFICATION, dtype: int64

In [138]:
application_type_counts = charity_df.APPLICATION_TYPE.value_counts()

In [139]:
# Determine which values to replace
replace_application_type = list(application_type_counts[application_type_counts < 700].index)
replace_application_type

['T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T15', 'T29', 'T17']

In [140]:
# Replace in DataFrame
for app_type in replace_application_type:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app_type,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [141]:
income_amt_counts = charity_df.INCOME_AMT.value_counts()
income_amt_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [142]:
# make a bin for 5m+
# Determine which values to replace
replace_income_amt = list(income_amt_counts[income_amt_counts < 300].index)
replace_income_amt

['10M-50M', '5M-10M', '50M+']

In [143]:
# Replace in DataFrame
for income in replace_income_amt:
    charity_df.INCOME_AMT = charity_df.INCOME_AMT.replace(income,"5M+")

# Check to make sure binning was successful
charity_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
5M+                564
10000-24999        543
Name: INCOME_AMT, dtype: int64

In [144]:
# Create a OneHotEncoder Instance
enc = OneHotEncoder(sparse = False)

In [145]:
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

In [146]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [147]:
# Merge one-hot encoded features and drop the orginals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,EIN,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_5M+
0,10520599,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10531628,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10547893,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10553066,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,10556103,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [148]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

In [149]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [150]:
# Create a StandardScaler instance.
scaler = StandardScaler()

In [151]:
# Fit the StandardScaler.
X_scaler = scaler.fit(X_train)

In [152]:
# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[-0.26741621, -0.03336766, -0.15558816, ..., -0.16981325,
         2.8419928 , -0.12694995],
       [ 0.90235315,  0.02868029, -0.15558816, ..., -0.16981325,
        -0.35186578,  7.87712001],
       [ 1.19104635, -0.03348439, -0.15558816, ..., -0.16981325,
        -0.35186578, -0.12694995],
       ...,
       [-1.15197331, -0.03348439, -0.15558816, ..., -0.16981325,
        -0.35186578, -0.12694995],
       [-0.03906835, -0.03348439, -0.15558816, ..., -0.16981325,
        -0.35186578, -0.12694995],
       [ 0.00827167, -0.03348439, -0.15558816, ..., -0.16981325,
        -0.35186578, -0.12694995]])

In [153]:
# Define the model.
number_input_features = len(X_train[0])
nn = tf.keras.models.Sequential()

In [154]:
# Add first and second hidden layers.
nn.add(
    tf.keras.layers.Dense(units=10, input_dim=number_input_features, activation="relu")
)
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))

In [155]:
# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [156]:
# Check the structure of the model.
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                400       
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 44        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 449
Trainable params: 449
Non-trainable params: 0
_________________________________________________________________


In [157]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [158]:
# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 2s 81us/sample - loss: 0.6243 - accuracy: 0.6719
Epoch 2/50
25724/25724 [==============================] - 1s 48us/sample - loss: 0.5629 - accuracy: 0.7283
Epoch 3/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5567 - accuracy: 0.7289
Epoch 4/50
25724/25724 [==============================] - 1s 57us/sample - loss: 0.5538 - accuracy: 0.7310
Epoch 5/50
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5518 - accuracy: 0.7315
Epoch 6/50
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5504 - accuracy: 0.7320
Epoch 7/50
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5495 - accuracy: 0.7324
Epoch 8/50
25724/25724 [==============================] - 1s 58us/sample - loss: 0.5484 - accuracy: 0.7325
Epoch 9/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5478 - accuracy: 0.7328
Epoch 10/50
25

In [159]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 23291387.4052 - accuracy: 0.4668
Loss: 23291387.405247815, Accuracy: 0.46682214736938477
